# Data collect and preprocessing


In [1]:
import wikipedia
import re
import os
import csv

import numpy as np
#from tqdm import tqdm
from tqdm.notebook import tqdm

In [2]:
import nltk 
from nltk.corpus import stopwords

In [4]:
CATEGORIES = ["sport", "politics", "economics", "psychology", "philosophy", "literature", "physics"]
DATASET_PATH = "./dataset/"
SENTENCE_LENGTH = 80

In [64]:
pag = wikipedia.page('mass')

In [65]:
print(pag.content)

A map is a symbolic depiction emphasizing relationships between elements of some space, such as objects, regions, or themes.
Many maps are static, fixed to paper or some other durable medium, while others are dynamic or interactive. Although most commonly used to depict geography, maps may represent any space, real or fictional, without regard to context or scale, such as in brain mapping, DNA mapping, or computer network topology mapping. The space being mapped may be two dimensional, such as the surface of the earth, three dimensional, such as the interior of the earth, or even more abstract spaces of any dimension, such as arise in modeling phenomena having many independent variables.
Although the earliest maps known are of the heavens, geographic maps of territory have a very long tradition and exist from ancient times. The word "map" comes from the medieval Latin Mappa mundi, wherein mappa meant napkin or cloth and mundi the world. Thus, "map" became a shortened term referring to 

In [55]:
pag = pag.content

In [56]:
pag = re.sub('===', ' ', pag)
pag = re.sub('==', ' ', pag)
pag = re.sub('=', ' ', pag)

In [59]:
print(pag)

In [16]:
def verify_or_create_dir(directory):
    if not os.path.isdir(directory):
        os.makedirs(directory)
        print("directory", directory, "created")
    else:
        print("directory", directory, "exist")

In [17]:
def get_art(category):
    #for cat in category:
    cat = category
    ric = wikipedia.search(cat)
    dir_name = DATASET_PATH + cat + "/"
    verify_or_create_dir(dir_name)
    for art in ric:
        try:
            pag = wikipedia.page(art)
            pag = pag.content
            pag = re.sub('===', ' ', pag)
            pag = re.sub('==', ' ', pag)
            pag = re.sub('=', ' ', pag)
            with open(dir_name + art + ".txt", 'w', encoding='UTF-8') as file:
                file.write(pag)
        except:
            print("Error in ", art)

In [18]:
def get_categories(cats):
    for cat in cats:
        get_art(cat)

In [19]:
get_categories(CATEGORIES)   # get_categories(CATEGORIES[2:])

directory ./dataset/sport/ created
Error in  Sport Chek
Error in  Swimming (sport)
directory ./dataset/politics/ created
directory ./dataset/economics/ created
Error in  Socioeconomics
Error in  Bachelor of Economics
directory ./dataset/psychology/ created
directory ./dataset/philosophy/ created
Error in  Indian philosophy
directory ./dataset/literature/ created
directory ./dataset/physics/ created
Error in  Physics


In [20]:
def save_dataset(categories, name_file):
    count = 0
    with open(DATASET_PATH + name_file + ".csv", 'w', newline='', encoding='UTF-8') as file_out:
        writer = csv.writer(file_out, delimiter= ',')
        for idx, cat in tqdm(enumerate(categories)):
            print("\n *** Importing", cat)
            path_cat = DATASET_PATH + cat + "/"
            files = os.listdir(path_cat)
            print(files)
            for f in files:
                with open(path_cat + f, 'r', encoding='UTF-8') as file_in:
                    file_text = file_in.read()
                    file_text = re.sub("\n", " ", file_text)
                    lst_sentences = file_text.split('. ')
                    row = ''
                    row_len = 0
                    for sentence in lst_sentences:
                        len_sent = len(sentence.split())
                        row += sentence
                        row_len += len_sent
                        if row_len + 9 > SENTENCE_LENGTH:
                            count += 1
                            writer.writerow([row, idx])
                            row = ''
                            row_len = 0
            print(count)
                    

In [21]:
save_dataset(CATEGORIES, "dataset_cat")


 *** Importing sport
['BBC Sport.txt', 'Bugatti Chiron.txt', 'Rowing (sport).txt', 'Sport (US magazine).txt', 'Sport diving (sport).txt', 'Sport of athletics.txt', 'Sport, Sport, Sport.txt', 'Sport.txt']
307

 *** Importing politics
['Ireland.txt', 'Political moderate.txt', 'Political party.txt', 'Political science.txt', 'Political spectrum.txt', 'Politics (Aristotle).txt', 'Politics of India.txt', 'Politics of the United States.txt', 'Politics.txt', 'Right-wing politics.txt']
758

 *** Importing economics
['Behavioral economics.txt', 'Economics.txt', 'Keynesian economics.txt', 'Labour economics.txt', 'London School of Economics.txt', 'Managerial economics.txt', 'Positive economics.txt', 'Service (economics).txt']
1174

 *** Importing psychology
['AP Psychology.txt', 'Cognitive psychology.txt', 'Developmental psychology.txt', 'Evolutionary psychology.txt', 'Flow (psychology).txt', 'Gestalt psychology.txt', 'Psyche (psychology).txt', 'Psychology.txt', 'Social psychology.txt', 'Transper

In [27]:
def noise_removal(text):
    text = re.sub('@', '', text)    
    text = re.sub('#', '', text)    
    text = re.sub('°', '', text)
    #text = re.sub('<[/A-Za-z]*>', '', text)
    text = re.sub('\...', '', text)
    return text

In [28]:
def normalization(text):
    text = text.replace(":-)", "smile")
    text = text.replace(":)", "smile")
    text = text.replace(":D", "smile")
    text = text.replace("b4", "before")
    text = text.replace("\n", " ")
    text = text.replace(",", " , ")
    text = text.replace(".", " . ")
    text = text.replace(":", " : ")
    text = text.replace(";", " ; ")
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.replace("  ", " ")    
    return text

In [22]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lpdepersiis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [23]:
def stopword_removal(text):
    new_words = []
    for word in text:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words
    

In [39]:
def preprocess_text(text):
    #print(text)
    text = text.lower()
    #print(text)
    text = normalization(text)
    #print(text)
    text = noise_removal(text)
    #print(text)
    text = text.split(' ')
    #text = nltk.word_tokenize(text)
    #print(text)
    text = stopword_removal(text)
    
    return text

In [58]:
text = "asfdaf, Risultati immagini per bloomberg health care efficiency * # The Bloomberg Health-Efficiency Index, first conducted in 2013, tracks life expectancy and medical spending to determine which health-care systems have the best outcomes."

In [44]:
#print(preprocess_text(text))

In [ ]:
print(noise_removal(text))

In [40]:
def get_one_hot(pos, len_vect):
    vec = np.zeros(len_vect)
    vec[int(pos)] = 1
    return vec

In [41]:
print(get_one_hot(2, 7))

[0. 0. 1. 0. 0. 0. 0.]


In [1]:
def get_x_y_preprocessed(name_file):
    x_rec = []
    y_rec = []
    with open(DATASET_PATH + name_file + ".csv", 'r', encoding='UTF-8') as file_dtset:
        reader = csv.reader(file_dtset, delimiter= ',')
        lines = [line for line in reader]
        #header = next(lines)
        # pbar = tqdm(lines)
        for row in tqdm(lines, total=len(lines)):
            try:
                text = row[0]
                text = preprocess_text(text)
                x_rec.append(text)
                y_rec.append(get_one_hot(row[1], len(CATEGORIES)))
                #print(type(y_rec))
                #print(type(x_rec))
            except:
                print("Exception:", row[0], row[1])
        y_rec = np.array(y_rec)
        
    return x_rec, y_rec


In [43]:
x_rec, y_rec = get_x_y_preprocessed("dataset_cat")

In [45]:
print(type(y_rec))

<class 'numpy.ndarray'>


In [46]:
print(len(x_rec))
print(y_rec.shape)

3341
(3341, 7)


In [47]:
print(y_rec)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
